# Less Naive representation pipeline (pennylane)

Main steps:

* input: image of the sky (frequency domain)
* apply 2D Fourier transform --> visibilities (Fourier domain)
* encode classical visibilities data into qbits (non-conventional domain)
* measure qbits (back to conventional domain)
* apply 2D Inverse Fourier transform --> original image?

In [2]:
import numpy as np
import math
import sys
#
import pennylane as qml
from pennylane.optimize import AdamOptimizer, GradientDescentOptimizer
from pennylane.templates.embeddings import AmplitudeEmbedding

## Generating an artificial image of the sky

In [4]:
#image of the sky filled with single precision complex floats
#pixels are set to low complex random values (image background) 
#few pixels are set to larger complex random values in a specified ellipse area (image source/subject)

n = 4
image = np.zeros((n, n), dtype=complex)
image.real = np.random.rand(n , n) / 100
image.imag = np.random.rand(n , n) / 100

h, w = image.shape
mask = circular_mask(h, w, radius=h/2)
sky_image = image.copy()
sky_image[~mask] = complex(np.random.rand() * 100, np.random.rand() * 100)
print(sky_image)

[[2.84653180e+01+5.57985789e+01j 2.84653180e+01+5.57985789e+01j
  5.56671510e-03+4.25495340e-03j 2.84653180e+01+5.57985789e+01j]
 [2.84653180e+01+5.57985789e+01j 2.83450815e-03+1.44317511e-03j
  3.29997304e-03+7.75675381e-03j 3.98866258e-03+8.48295479e-03j]
 [4.49326327e-03+9.01956359e-03j 7.24373391e-03+1.56689646e-03j
  6.17452219e-03+4.51461759e-03j 7.38879222e-03+9.80845703e-03j]
 [2.84653180e+01+5.57985789e+01j 7.86494639e-03+7.26794462e-04j
  2.21786548e-03+6.67424981e-03j 3.84445068e-03+9.42341614e-03j]]


## Applying 2D Fourier transform (visibilities)

In [5]:
visibilities = np.fft.fft2(sky_image)
print(visibilities)

[[142.38150758+279.05656628j  85.35921031+167.37883438j
   28.45390527 +55.79934733j  85.40716624+167.38427694j]
 [ 85.37707891+167.37888621j  28.47376628 +55.7890992j
  -28.45447999 -55.8024511j   28.44693386 +55.792703j  ]
 [ 28.47213465 +55.79323603j -28.45955332 -55.78088649j
  -85.38133738-167.38367755j -28.47454301 -55.78690929j]
 [ 85.37536207+167.37127797j  28.46558199 +55.79024867j
  -28.45709316 -55.79051443j  28.45944817 +55.7872251j ]]


### Sanity check

In [6]:
img = np.fft.ifft2(visibilities)
print(img)
test_real = ((sky_image.real - img.real)**2).mean()
test_imag = ((sky_image.imag - img.imag)**2).mean()

print()
 
print('Reals MSE: '+ str(test_real))
print('Imaginaries MSE: ' + str(test_imag))

[[2.84653180e+01+5.57985789e+01j 2.84653180e+01+5.57985789e+01j
  5.56671510e-03+4.25495340e-03j 2.84653180e+01+5.57985789e+01j]
 [2.84653180e+01+5.57985789e+01j 2.83450815e-03+1.44317511e-03j
  3.29997304e-03+7.75675381e-03j 3.98866258e-03+8.48295479e-03j]
 [4.49326327e-03+9.01956359e-03j 7.24373391e-03+1.56689646e-03j
  6.17452219e-03+4.51461759e-03j 7.38879222e-03+9.80845703e-03j]
 [2.84653180e+01+5.57985789e+01j 7.86494639e-03+7.26794462e-04j
  2.21786548e-03+6.67424981e-03j 3.84445068e-03+9.42341614e-03j]]

Reals MSE: 7.213723938185513e-30
Imaginaries MSE: 1.7278693134916627e-29


## Classical data encoding/decoding

In [7]:
# Amplitude embedding encodes a normalized 2^n-dimensional feature vector into the state of n qbits (uses log2(n) qbits for n classical data)
data_dimension = visibilities.shape[0]*visibilities.shape[1]
data = visibilities.flatten()

#normalization to prepare a qstate with measurement probabilites summing up to 1 (SUM (amplitudes²) = 1)
norm = qml.math.sum(qml.math.abs(data) ** 2)
normalized_data = data / qml.math.sqrt(norm)

wires = range(int(math.log2(data_dimension))) # set the number of qbits (no padding needed if outputs an integer)
amp_dev = qml.device('default.qubit', wires, shots=1)

@qml.qnode(amp_dev)
def amp_encoding(data):
    AmplitudeEmbedding(data, wires) # normalize = True
    return qml.sample(qml.PauliX(0)), qml.sample(qml.PauliX(1)), qml.sample(qml.PauliX(2)), qml.sample(qml.PauliX(3))

result = amp_encoding(normalized_data)
print(result)
readout = amp_dev.state

readout = readout*qml.math.sqrt(norm) # denormalization of the measurement outcomes
readout = np.array(readout).reshape(n , n)
print(readout)

[[ 1]
 [ 1]
 [-1]
 [ 1]]
[[142.38150758+279.05656628j  85.35921031+167.37883438j
   28.45390527 +55.79934733j  85.40716624+167.38427694j]
 [ 85.37707891+167.37888621j  28.47376628 +55.7890992j
  -28.45447999 -55.8024511j   28.44693386 +55.792703j  ]
 [ 28.47213465 +55.79323603j -28.45955332 -55.78088649j
  -85.38133738-167.38367755j -28.47454301 -55.78690929j]
 [ 85.37536207+167.37127797j  28.46558199 +55.79024867j
  -28.45709316 -55.79051443j  28.45944817 +55.7872251j ]]


## Applying 2D Inverse Fourier transform (+ fidelity test)

In [8]:
img = np.fft.ifft2(readout)

test_real = ((sky_image.real - img.real)**2).mean()
test_imag = ((sky_image.imag - img.imag)**2).mean()

print(test_real.dtype)

print('Reals MSE: '+ str(test_real))
print('Imaginaries MSE: ' + str(test_imag))

float64
Reals MSE: 5.827872898453068e-30
Imaginaries MSE: 1.4526298503144218e-29


## Utils

### Quantum

### Classical

In [3]:
#creates a circular mask over a 2D array
def circular_mask(h, w, center=None, radius=None):
    if center is None: #image center
        center = (int(w/2), int(h/2))
    if radius is None: #smallest distance between center and image bounderies
        radius = min(center[0], center[1], w-center[0], h-center[1])
        
    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)
    mask = dist_from_center <= radius
    
    return mask